In [30]:
eyeear = '/Users/mz/Downloads/135562304_new-york-eye-and-ear-infirmary-of-mount-sinai_standardcharges.json'
stlukes = '/Users/mz/Downloads/132997301_mount-sinai-morningside_standardcharges.json'
south = '/Users/mz/Downloads/111352310_mount-sinai-south-nassau_standardcharges.json'

In [32]:
import ijson
import polars as pl
import json

In [33]:
with open(south , 'r') as file:
    df = json.load(file)
cl = pl.from_dicts(df['standard_charge_information'])
cl = (
    cl
    # 1: explode standard_charges (list → rows)
    .explode("standard_charges")
    .unnest("standard_charges")

    # 3: code_information is a list of 1 element → flatten
    .explode("code_information")
    .unnest("code_information")
)

In [34]:
cpt = (cl.filter(pl.col('code') == '92250')
       .with_columns([
           pl.lit('').alias('methodology'),
           pl.lit('St Lukes').alias('setting'),
           pl.lit('All Payers').alias('payer_name'),
           pl.lit('All Plans').alias('plan_name'),
           pl.col('discounted_cash').cast(pl.Float64).alias('standard_charge_dollar'),
       ])
)

In [35]:
(cpt[['setting', 'payer_name', 'plan_name', 'standard_charge_dollar', 'methodology']]).write_csv(f"south_92250_cpt.csv")

In [36]:
cpt

description,code,type,gross_charge,discounted_cash,setting,billing_class,additional_generic_notes,methodology,payer_name,plan_name,standard_charge_dollar
str,str,str,f64,f64,str,str,str,str,str,str,f64
